In [4]:
!pip install pandas scikit-learn numpy sentence-transformers tabulate
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import timeit 

try:
    MODEL = SentenceTransformer('all-MiniLM-L6-v2')
    print("Sentence Transformer Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

# data prep
data = [
    {"name": "Boho Maxi Dress", "desc": "Flowy, lightweight linen with earthy tones and floral embroidery. Perfect for a music festival or beach walk.", "vibe_tags": ["boho", "relaxed", "festival"]},
    {"name": "Urban Tech Joggers", "desc": "Slim-fit, black technical fabric with reflective strips and zippered pockets. Functional and edgy city style.", "vibe_tags": ["urban", "edgy", "techwear"]},
    {"name": "Chunky Knit Cardigan", "desc": "Oversized, soft, chunky wool, cream-colored, ideal for a comfortable night in or casual weekend coffee date.", "vibe_tags": ["cozy", "casual", "hygge"]},
    {"name": "Power Shoulder Blazer", "desc": "Structured, sharp shoulders, deep charcoal grey. Commands authority, perfect for professional meetings and confidence.", "vibe_tags": ["professional", "chic", "sharp"]},
    {"name": "Retro Neon Sneakers", "desc": "Bright neon colors, chunky platform sole, 90s inspired design. Great for a bold, energetic athletic look.", "vibe_tags": ["energetic", "retro", "sporty"]},
    {"name": "Minimalist Silk Scarf", "desc": "Pure white silk, clean lines, simple and elegant. A refined accessory for a quiet luxury look.", "vibe_tags": ["minimalist", "elegant", "luxury"]},
    {"name": "Vintage Denim Jacket", "desc": "Worn-in blue denim with slight fading and brass buttons. A rugged, timeless look for everyday wear.", "vibe_tags": ["vintage", "casual", "rugged"]},
    {"name": "Sleek Cocktail Dress", "desc": "Deep crimson satin, knee-length, fitted silhouette. Perfect for a sophisticated evening event or date.", "vibe_tags": ["formal", "sophisticated", "elegant"]},
]
df = pd.DataFrame(data)

def get_embedding(text: str) -> list[float]:
    embedding = MODEL.encode(text)
    return embedding.tolist()

print(f"Total Products: {len(df)}. Generating local embeddings...")
df['description_embedding'] = df['desc'].apply(get_embedding)
print("Product embeddings generated and stored successfully")

Sentence Transformer Model loaded successfully.
Total Products: 8. Generating local embeddings...
Product embeddings generated and stored successfully


In [6]:
def find_vibe_matches(df_data: pd.DataFrame, query_text: str, n_top: int = 3, threshold: float = 0.5) -> tuple[pd.DataFrame, str]:
    query_embedding = get_embedding(query_text)
    product_embeddings_matrix = np.array(df_data['description_embedding'].tolist())
    query_embedding_array = np.array(query_embedding).reshape(1, -1)
    similarity_scores = cosine_similarity(query_embedding_array, product_embeddings_matrix)[0]
    df_data['similarity_score'] = similarity_scores.copy() 
    top_matches = df_data.sort_values(by='similarity_score', ascending=False).head(n_top).copy()
    top_score = top_matches.iloc[0]['similarity_score'] if not top_matches.empty else 0
    
    fallback_message = "Valid"
    if top_score < threshold:
        fallback_message = f"Below Threshold ({threshold}): Fallback Prompt activated."
        
    return top_matches[['name', 'desc', 'vibe_tags', 'similarity_score']], fallback_message

queries = {
    "Query 1 (Assignment)": "energetic urban chic",
    "Query 2 (Relaxed)": "What would I wear to feel relaxed and warm on a cold night?",
    "Query 3 (Professional)": "I need a refined look for a big client presentation."
}
all_results = {}

for name, q in queries.items():
    print(f"\nRunning {name}: '{q}'")
    results_df, edge_case_status = find_vibe_matches(df, q)
    all_results[name] = {'df': results_df, 'edge_case': edge_case_status, 'query': q}
    print(results_df)
    print(f"Edge Case Status: {edge_case_status}")

print("\n" + "="*70)

GOOD_MATCH_THRESHOLD = 0.7 
metrics = []

for name, data in all_results.items():
    top_score = data['df']['similarity_score'].iloc[0]
    is_good = "Yes" if top_score > GOOD_MATCH_THRESHOLD else "No"
    
    metrics.append({
        "Test": name,
        "Top Match Name": data['df']['name'].iloc[0],
        "Top Score": f"{top_score:.4f}",
        f"Good Match? (Score > {GOOD_MATCH_THRESHOLD})": is_good,
        "Edge Case Check": data['edge_case']
    })

metrics_df = pd.DataFrame(metrics)
print("\nPerformance Metrics (Accuracy/Eval)")
print(metrics_df.to_markdown(index=False))

print("\nLatency Measurement (Local CPU)")

search_latency = timeit.timeit(lambda: find_vibe_matches(df, queries['Query 1 (Assignment)'], n_top=1), number=1)
print(f"Total Vibe Matcher Latency (Full Search): {search_latency*1000:.4f} milliseconds")

print("\n" + "="*70)


Running Query 1 (Assignment): 'energetic urban chic'
                  name                                               desc  \
1   Urban Tech Joggers  Slim-fit, black technical fabric with reflecti...   
4  Retro Neon Sneakers  Bright neon colors, chunky platform sole, 90s ...   
0      Boho Maxi Dress  Flowy, lightweight linen with earthy tones and...   

                    vibe_tags  similarity_score  
1     [urban, edgy, techwear]          0.241910  
4  [energetic, retro, sporty]          0.238622  
0   [boho, relaxed, festival]          0.181544  
Edge Case Status: Below Threshold (0.5): Fallback Prompt activated.

Running Query 2 (Relaxed): 'What would I wear to feel relaxed and warm on a cold night?'
                   name                                               desc  \
2  Chunky Knit Cardigan  Oversized, soft, chunky wool, cream-colored, i...   
7  Sleek Cocktail Dress  Deep crimson satin, knee-length, fitted silhou...   
0       Boho Maxi Dress  Flowy, lightweight l